# Scheduling

In this notebook, we explore how to solve a resource constrained project scheduling problem (RCPSP).

The problem is made of $M$ activities that have precedence constraints. That means that if activity $j\in[1,M]$ is a successor of activity $i\in[1,M]$, then activity $i$ must be completed before activity $j$ can be started

On top of these constraints, each project is assigned a set of K renewable resources where each resource $k$ is available in $R_{k}$ units for the entire duration of the project. Each activity may require one or more of these resources to be completed. While scheduling the activities, the daily resource usage for resource $k$ can not exceed $R_{k}$ units.

Each activity $j$ takes $d_{j}$ time units to complete.

The overall goal of the problem is usually to minimize the makespan,

(Variants: mix of renewable and non-renewable resources / multi-mode)


In [ ]:
import sys, os
sys.path.append('../')

from skdecide.discrete_optimization.rcpsp.parser.rcpsp_parser import parse_file
#from skdecide.discrete_optimization.rcpsp.parser.rcpsp_parser import files_available, get_data_available
#from skdecide.discrete_optimization.rcpsp.rcpsp_model import plt, MethodBaseRobustification
from skdecide.discrete_optimization.rcpsp.rcpsp_model import RCPSPModel, SingleModeRCPSPModel, MultiModeRCPSPModel, \
RCPSPSolution, create_poisson_laws, UncertainRCPSPModel, Aggreg_RCPSPModel, MethodAggregating, \
MethodRobustification

## Single mode RCPSP
Each activity has only one mode of execution, we can ignore the fact that there are renewable or unrenewable resources in this case. Let's look at one instance of an RCPSP, this is `j301_1.sm` file that you can retrieve in the data folder.

## Parsing the text file in DO-lib

In [ ]:
#from skdecide.discrete_optimization.rcpsp import compute_graph_rcpsp, Graph

from skdecide.discrete_optimization.rcpsp.rcpsp_utils import RCPSPModel
from skdecide.discrete_optimization.rcpsp.parser.rcpsp_parser import parse_file
#get_data_available
#
import networkx as nx
import matplotlib.pyplot as plt

file = 'j301_1.sm'

rcpsp_problem = parse_file(file)
print("Nb jobs : ", rcpsp_problem.n_jobs)
print("Precedences : ", rcpsp_problem.successors)
print("Resources Availability : ", rcpsp_problem.resources)
#print("Details : ", rcpsp_problem.mode_details)

### Look at precedence graph

In [ ]:
from skdecide.builders.domain.scheduling.graph_toolbox import Graph
from skdecide.discrete_optimization.rcpsp.rcpsp_model import compute_graph_rcpsp

graph: Graph = compute_graph_rcpsp(rcpsp_problem)

In [ ]:
graph_nx = graph.to_networkx()
dfs = nx.dfs_tree(G=graph_nx, source=1, depth_limit=10)
shortest_path_length = nx.shortest_path_length(dfs, 1)
length_to_nodes = {}
position = {}
for node in sorted(shortest_path_length, key=lambda x: shortest_path_length[x]):
    length = shortest_path_length[node]
    while not(length not in length_to_nodes or len(length_to_nodes[length]) <= 5):
        length += 1
    if length not in length_to_nodes:
        length_to_nodes[length] = []
    length_to_nodes[length] += [node]
    position[node] = (length, len(length_to_nodes[length]))
nx.draw_networkx(graph_nx, pos=position)
plt.show()

### Critical path 
We can compute the largest path possible from source to sink task, it gives a lower bound on the makespan. 
When we computed the graph in previous cell, each edges store the minimum duration of a task, we also store the opposite of this number in ```minus_min_duration``` attribute of an edge.

In [ ]:
print(graph.edges[5])

This means to fulfill the (2, 15) precedence you have to accomplish the task 2, which takes minimum 8 unit times to do. Let's compute the critical path. 

In [ ]:
path = nx.astar_path(G=graph_nx,
                     source=1,
                     target=rcpsp_problem.n_jobs+2,
                     heuristic=lambda x, y: -100,
                     weight="minus_min_duration")
edges = [(n1, n2) for n1, n2 in zip(path[:-1], path[1:])]
duration = sum(graph_nx[n[0]][n[1]]["min_duration"] for n in edges)
print("Duration of critical path : ", duration)

We know that our makespan will be at minimum 18 then.

In [ ]:
fig, ax = plt.subplots(1)
nx.draw_networkx(graph_nx, pos=position, ax=ax)
nx.draw_networkx_edges(graph_nx, pos=position, edgelist=edges, edge_color="r", ax=ax)

## Dummy solution
A solution can be defined as a permutation of jobs which is then transformed into a feasible schedule if possible using the SGS routine. It consists at scheduling an activity as soon as it is available following the permutation order if possible.

In [ ]:
permutation = [25, 0, 19, 29, 21, 27, 18, 15, 28, 14, 26, 3, 17, 9, 
               24, 16, 13, 8, 1, 6, 10, 20, 7, 11, 4, 2, 5, 22, 12, 23]
rcpsp_model = rcpsp_problem
mode_list = [1 for i in range(rcpsp_model.n_jobs)]
rcpsp_sol = RCPSPSolution(problem=rcpsp_model, rcpsp_permutation=permutation, rcpsp_modes=mode_list)
print('schedule feasible: ', rcpsp_sol.rcpsp_schedule_feasible)
print('schedule: ', rcpsp_sol.rcpsp_schedule)
print('rcpsp_modes:', rcpsp_sol.rcpsp_modes)
fitnesses = rcpsp_model.evaluate(rcpsp_sol)
print('fitnesses: ', fitnesses)
resource_consumption = rcpsp_model.compute_resource_consumption(rcpsp_sol)
print('resource_consumption: ', resource_consumption)
print('mean_resource_reserve:', rcpsp_sol.compute_mean_resource_reserve())

In [ ]:
from skdecide.discrete_optimization.rcpsp.rcpsp_plot_utils import plot_resource_individual_gantt, plot_ressource_view

#### Plotting the recomputed schedule

In [ ]:
plot_resource_individual_gantt(rcpsp_model=rcpsp_model,
                               rcpsp_sol=rcpsp_sol,
                               title_figure="Dummy solution")

## First greedy solution : The RCPSP Pile solver

First idea is to iteratively build a schedule from source to sink, considering available task at each time, and choosing among the available task with a greedy objective.
A quite clever greedy choice is to use the graph structure of the precedence graph. We consider that task that have a lot of successors state in the graph is more important than the others : indeed it means that doing this task will unlock more following tasks. 
That's what the greedy solver is doing :

In [ ]:
from skdecide.discrete_optimization.rcpsp.rcpsp_solvers import PileSolverRCPSP, GreedyChoice, ResultStorage

In [ ]:
solver = PileSolverRCPSP(rcpsp_model=rcpsp_model)
result_storage = solver.solve(greedy_choice=GreedyChoice.MOST_SUCCESSORS)

In [ ]:
best_solution = result_storage.get_best_solution()
fitnesses = rcpsp_model.evaluate(best_solution)
print('fitnesses: ', fitnesses)

In [ ]:
plot_ressource_view(rcpsp_model=rcpsp_model,
                    rcpsp_sol=best_solution,
                    title_figure="Pile solution")
plot_resource_individual_gantt(rcpsp_model=rcpsp_model,
                               rcpsp_sol=best_solution,
                               title_figure="Pile solution")

In fact for this instance, this schedule is optimal on makespan. 
Good luck for your next notebooks !

## WIP : run solver of your choice easily

In [ ]:
from discrete_optimization.rcpsp.rcpsp_solvers import *

Let's list all solvers available. It is not complete since it doesn't contain the LS and GA algorithm (to be done !)

In [ ]:
print(solvers.keys())
solvers_option = [(k, submethod) for k in solvers for submethod in solvers[k]]
names_solver = [so[1][0].__name__ for so in solvers_option]
print(names_solver)

In [ ]:
index_to_choose = names_solver.index("LP_RCPSP") # you can choose any solver in the names_solver list. 
solution = solve(method=solvers_option[index_to_choose][1][0],
                 rcpsp_model=rcpsp_model,
                 **solvers_option[index_to_choose][1][1])

In [ ]:
best_solution = solution.get_best_solution()
fitnesses = rcpsp_model.evaluate(best_solution)
print('fitnesses: ', fitnesses)

# WIP: Use othe viz library

In [ ]:
%%javascript
// Import eCharts, the Javascript charting library
require.config({
    paths: {
        ech: 'https://raw.githubusercontent.com/frappe/gantt/master/dist/frappe-gantt.min.js'
    }
});

In [ ]:
rcpsp_model.mode_details.keys()

In [ ]:
rcpsp_sol.rcpsp_schedule

In [ ]:
class Task:
    def __init__(self, id, start_time, end_time, dependecies):
        self.id = id
        self.start_time = start_time
        self.end_time = end_time
        self.dependencies = dependecies

In [ ]:
tasks = []
for task in rcpsp_sol.rcpsp_schedule:
    tasks.append(Task(task, rcpsp_sol.rcpsp_schedule[task]['start_time'], rcpsp_sol.rcpsp_schedule[task]['end_time'],[]))

In [ ]:
import json

json_string = json.dumps([t.__dict__ for t in tasks])

In [ ]:
json_string